In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import datetime
import statsmodels.formula.api as smf
from bson import ObjectId
from pymongo import MongoClient
import matplotlib as mpl

/Users/xiaoyu/miniforge3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
client = MongoClient('127.0.0.1', 27017)
db = client.moniter_crowdfunding

In [4]:
project = db.projects
s_project = db.success_projects
f_project = db.failure_projects
front_page = db.front_page
company = db.companies

In [5]:
x = s_project.find_one({}, projection={'项目动态信息': False, '各档动态信息': False, '评论': False})
x.keys()

dict_keys(['_id', '众筹期限', '公司地址', '状态变换时间1-2', '各档基础信息', '爬取次数', '项目名称', '所属类别', '状态', '公司电话', '状态变换时间0-1', '目标金额', '发起人链接', '公司工作时间', '公司名称', '发起人名称', '状态变换时间2-3', '状态变换时间3-4'])

In [6]:
def get_front_demo_data(a_dict, t_dict, pid):
    """
    递归获取时间序列t_dict。t_dict与a_dict键相对应，查看对应时期各监测时间点的各展区是否包含目标pid
    """
    for key in a_dict:
        if isinstance(a_dict[key], list):
            if pid in a_dict[key]:
                t_dict[key] = t_dict.get(key, []) + [1]
            else:
                t_dict[key] = t_dict.get(key, []) + [0]
        elif isinstance(a_dict[key], dict):
            t_dict[key] = get_front_demo_data(a_dict[key], {}, pid)
        elif key == "监测时间":
            t_dict[key] = t_dict.get(key, []) + [a_dict[key]]
        else:
            pass
    return t_dict

In [14]:
def get_eff_data(t_dict, prev_keys=[], dataset={}):
    """
    递归获取t_dict对应压缩形式
    """
    for key in t_dict:
        if isinstance(t_dict[key], list):
            c_key = ">".join(prev_keys + [key]) if prev_keys else key
            dataset[c_key] = t_dict[key]
        else:
            dataset = get_eff_data(
                t_dict[key], prev_keys + [key], dataset)
            
    return dataset

In [18]:
def get_front_position(proj):
    """
    获取众筹生命周期内在首页各位置出现时间的判断
    """
    # 项目扩散数据最早与最晚时间点
    t_start = proj['项目动态信息'][0]['爬取时间']
    t_end = proj['项目动态信息'][-1]['爬取时间'] + \
        datetime.timedelta(hours=8)
    # 取出这个时间段的主页观察数据
    f_page = front_page.find(
        {"监测时间": {"$gte": t_start, "$lte": t_end}}, sort=[("监测时间", 1)]
    )
    # 迭代查看_id是否存在各时间点的各展区
    t_dict = {}
    for a in f_page:
        t_dict = get_front_demo_data(a, t_dict, id)
        
    # 转换为DataFrame
    dataset = get_eff_data(t_dict)
    col_names = list(dataset)
    col_names.remove("监测时间")
    front_df = pd.DataFrame(
        dataset, index=dataset["监测时间"], columns=col_names)
    
    return front_df

In [20]:
x = s_project.find_one()

In [21]:
front_df = get_front_position(x)

In [12]:
pid = '98553'
t_dict = {}
f_item = front_page.find_one({})
t_dict = get_front_demo_data(f_item, t_dict=t_dict, pid=pid)

In [13]:
t_dict

{'轮播': {'r-t': [0], 'left': [0], 'r-b-r': [0], 'r-b-l': [0]},
 '即将上架': [1],
 'demostr_proj': {'健康出行': {'左上大图': [0], '其它图': [0]},
  '生活美学': {'左上大图': [0], '其它图': [0]},
  '文化艺术': {'左上大图': [0], '其它图': [0]},
  '惠民扶贫': {'左上大图': [0], '其它图': [0]},
  '美食市集': {'左上大图': [0], '其它图': [0]},
  '新奇酷玩': {'左上大图': [0], '其它图': [0]}},
 '监测时间': [datetime.datetime(2018, 4, 28, 10, 10, 55, 130000)],
 '即将结束': [0],
 '热门推荐': [0],
 '最新上架': [0]}

In [ ]:
get_eff_data(t_dict=t_dict)

{'轮播>r-t': [0],
 '轮播>left': [0],
 '轮播>r-b-r': [0],
 '轮播>r-b-l': [0],
 '即将上架': [1],
 'demostr_proj>健康出行>左上大图': [0],
 'demostr_proj>健康出行>其它图': [0],
 'demostr_proj>生活美学>左上大图': [0],
 'demostr_proj>生活美学>其它图': [0],
 'demostr_proj>文化艺术>左上大图': [0],
 'demostr_proj>文化艺术>其它图': [0],
 'demostr_proj>惠民扶贫>左上大图': [0],
 'demostr_proj>惠民扶贫>其它图': [0],
 'demostr_proj>美食市集>左上大图': [0],
 'demostr_proj>美食市集>其它图': [0],
 'demostr_proj>新奇酷玩>左上大图': [0],
 'demostr_proj>新奇酷玩>其它图': [0],
 '监测时间': [datetime.datetime(2018, 4, 28, 10, 10, 55, 130000)],
 '即将结束': [0],
 '热门推荐': [0],
 '最新上架': [0]}